# Preparation

In [ ]:
import torch
torch.__version__

In [ ]:
import os
import pandas as pd
import numpy as np
import random
from torch import nn

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from model import *
from dataset import *
from train import *

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
BATCH_SIZE = 256
NUMEPOCHS = 50

In [ ]:
# import pandas as pd
# df = pd.read_csv("Data/ml-1m.train.rating",sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'], engine='python')
# df

In [ ]:
# df2 = pd.read_csv("Data/ml-1m.test.negative",sep = "\t",header=None)
# df2

# Trian and Test

### Compare the three methods 

In [ ]:
# model_config_mf = {
#     "model_mark": "gmf(mf_dim=8)",
#     'embedding_dim_mf': 8,
#     # "mlp_layers(X)" : 0,
#     # 'mlp_layers': [ 32, 16, 8],
#     'model_type': 'GMF'     #　MLP, NeuMF
# }

# model_config_mlp = {
#     "model_mark": "mlp(mlp_layer=3)",
#     "mlp_layers(X)" : 3,
#     'mlp_layers': [64, 32, 16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_neumf = {
#     "model_mark": "neumf(mf_dim=8,mlp_layer=3)",
#     'embedding_dim_mf': 8,
#     "mlp_layers(X)" : 3,
#     'mlp_layers': [64, 32, 16, 8],
#     'model_type': 'NeuMF'     #　MLP, NeuMF
# }

# model_train(model_config_mf, seed = 42, num_of_negatives=4, num_of_epochs = 50)


In [ ]:
# model_train(model_config_mlp, seed = 42, num_of_negatives=4, num_of_epochs=30)


In [ ]:
# model_train(model_config_neumf, seed = 42, num_of_negatives=4, num_of_epochs=30)


for movielens

HR, NDCG

-   GMF(Factor-8, X-0): 0.6339, 0.3621
-   MLP(Factor-8, X-3): 0.6109, 0.3434
-   NeuMF(Factor-8, X-3): 0.6474, 0.3731

## Reproduce the ablation study --- MLP with different layers

 K: 8
 
 layers (X): 0, 1, 2, 3, 4

In [ ]:
# model_config_mlp0 = {
#     "model_mark": "mlp(mlp_layer=0)",
#     "mlp_layers(X)" : 0,
#     'mlp_layers': [8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp1 = {
#     "model_mark": "mlp(mlp_layer=1)",
#     "mlp_layers(X)" : 1,
#     'mlp_layers': [16,8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# #! [8][:-1],[8][1:]    # ([], [])
# #! [8,16][:-1],[8,16][1:]  # ([8], [16])

# model_config_mlp2 = {
#     "model_mark": "mlp(mlp_layer=2)",
#     "mlp_layers(X)" : 2,
#     'mlp_layers': [32,16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp3 = {
#     "model_mark": "mlp(mlp_layer=3)",
#     "mlp_layers(X)" : 3,
#     'mlp_layers': [64,32, 16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp4 = {
#     "model_mark": "mlp(mlp_layer=4)",
#     "mlp_layers(X)" : 4,
#     'mlp_layers': [128,64, 32, 16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_train(model_config_mlp0, seed = 42, num_of_negatives=4, num_of_epochs=30)


In [ ]:
# model_train(model_config_mlp1, seed = 42, num_of_negatives=4, num_of_epochs=30)


In [ ]:
# model_train(model_config_mlp2, seed = 42, num_of_negatives=4, num_of_epochs=30)


In [ ]:
# model_train(model_config_mlp3, seed = 42, num_of_negatives=4, num_of_epochs=30)


In [ ]:
# model_train(model_config_mlp4, seed = 42, num_of_negatives=4, num_of_epochs=30)


HR, NDCG

-   MLP0: 0.4533, 0.2521
-   MLP1: 0.4907, 0.2731
-   MLP2: 0.5104, 0.2847
-   MLP3: 0.5667, 0.3159
-   MLP4: 0.6053, 0.3423

# Plot